In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================
# 1️⃣ Load source dataset
# ============================================================
source_path = Path("output_datasets/motorbike_final_dataset_clean.csv")
df = pd.read_csv(source_path)

print(f"✅ Loaded source dataset: {df.shape}")

# ============================================================
# 2️⃣ Recreate mean log-price features
# ============================================================
if "Log_Gia" not in df.columns:
    df["Log_Gia"] = np.log1p(df["Gia"])

df["Brand_mean_price"] = df["Thuong_hieu_code"].map(df.groupby("Thuong_hieu_code")["Log_Gia"].mean())
df["Dong_mean_price"] = df["Dong_xe_code"].map(df.groupby("Dong_xe_code")["Log_Gia"].mean())
df["Segment_mean_price"] = df["Phan_khuc_dung_tich_code"].map(df.groupby("Phan_khuc_dung_tich_code")["Log_Gia"].mean())

global_mean = df["Log_Gia"].mean()
df[["Brand_mean_price", "Dong_mean_price", "Segment_mean_price"]] = (
    df[["Brand_mean_price", "Dong_mean_price", "Segment_mean_price"]].fillna(global_mean)
)

print("✅ Added mean log-price features successfully.")

# ============================================================
# 3️⃣ Save reference dataset
# ============================================================
output_path = Path("output_datasets/motorbike_final_dataset_ref.csv")
df.to_csv(output_path, index=False)
print(f"💾 Saved reference dataset to {output_path}")


✅ Loaded source dataset: (7208, 15)
✅ Added mean log-price features successfully.
💾 Saved reference dataset to output_datasets\motorbike_final_dataset_ref.csv


In [3]:
import zipfile, os

zip_path = "output_datasets/model_files.zip"
with zipfile.ZipFile(zip_path, "w") as z:
    z.write("output_datasets/best_model_XGBoost.pkl", arcname="best_model_XGBoost.pkl")
    z.write("output_datasets/scaler_XGBoost.pkl", arcname="scaler_XGBoost.pkl")

print(f"✅ Created {zip_path}, size = {os.path.getsize(zip_path)/1024/1024:.2f} MB")


✅ Created output_datasets/model_files.zip, size = 3.90 MB


In [4]:
import zipfile
with zipfile.ZipFile("output_datasets/model_files.zip", "w") as z:
    z.write("output_datasets/best_model_XGBoost.pkl", arcname="best_model_XGBoost.pkl")
    z.write("output_datasets/scaler_XGBoost.pkl", arcname="scaler_XGBoost.pkl")


In [8]:
model = joblib.load("output_datasets/best_model_XGBoost.pkl")
scaler = joblib.load("output_datasets/scaler_XGBoost.pkl")
df = pd.read_csv("output_datasets/motorbike_final_dataset_clean.csv")

X = df[FEATURES]
X[scale_cols] = scaler.transform(X[scale_cols])
y = df["Log_Gia"]
y_pred = model.predict(X)

print(r2_score(y, y_pred))


KeyError: "['Brand_mean_price', 'Dong_mean_price', 'Segment_mean_price'] not in index"